In [1]:
import os
import pandas as pd
from dotenv import load_dotenv

load_dotenv()

RETRIEVE_BASE_URL = os.getenv("RETRIEVE_BASE_URL")
RETRIEVE_ENDPOINT = os.getenv("RETRIEVE_ENDPOINT")
RETRIEVE_BASE_URL

'http://10.204.100.79:2323'

In [ ]:
data = pd.read_csv('data/30012026_MultiRAG_val_set.csv')[:100]
data

,conver_id,pattern,sampling_stergy,turn_ids,questions,answers,facts
0,10,CCCC,1,1,ลูกมีปัญหาในการอ่านและเขียนผิดพลาดบ่อย ควรเริ่...,ควรเริ่มต้นด้วยการปรึกษาจิตแพทย์เฉพาะทางด้านคว...,[{'title': 'การจัดการภาวะบกพร่องด้านการเรียนรู...
1,10,CCCC,1,2,การตรวจอัลตราซาวด์เต้านมสามารถใช้ตรวจพบภาวะใดไ...,การตรวจอัลตราซาวด์เต้านมสามารถใช้ตรวจพบภาวะต่า...,[{'title': 'อัลตราซาวนด์เต้านมมีบทบาทหลักในการ...
2,10,CCCC,1,3,โรคงูสวัดเกิดจากเชื้อไวรัสชนิดใดที่กลับมาทำให้...,โรคงูสวัดเกิดจากเชื้อไวรัสวาริเซลลา-โซสเตอร์ (...,[{'title': 'โรคงูสวัดคือผื่นและตุ่มน้ำใสที่เกิ...
3,10,CCCC,1,4,อาการปวดศีรษะเรื้อรังสามารถเกี่ยวข้องกับภาวะหล...,อาการปวดศีรษะเรื้อรังสามารถเกี่ยวข้องกับภาวะหล...,[{'title': 'อาการสำคัญของโรคหลอดเลือดอักเสบจาก...
4,10,CCCC,1,5,ทารกแรกเกิดที่มีภาวะช็อกอาจแสดงอาการอย่างไรในช...,ทารกแรกเกิดที่มีภาวะช็อกอาจแสดงอาการอัตราการเต...,[{'title': 'โดยทั่วไปความดันโลหิตสูงในทารกแรกเ...
...,...,...,...,...,...,...,...
95,320,DRPP,1,4,มีปัจจัยใดบ้างที่ทำให้ไอหลังเป็นหวัดยืดเยื้อเก...,ปัจจัยที่ทำให้ไอหลังเป็นหวัดยืดเยื้อเกิน 3 สัป...,[{'title': 'ไอกึ่งเฉียบพลันคืออาการไอที่มีระยะ...
96,320,DRPP,1,5,อะไรคือสาเหตุหลักที่ทำให้ไอหลังเป็นหวัดไม่หายเ...,สาเหตุหลักที่ทำให้ไอหลังเป็นหวัดไม่หายเสียทีคื...,[{'title': 'ภาวะไอหลังเป็นหวัดเกิดจากการอักเสบ...
97,327,DDDD,1,1,อาการใดที่แสดงว่าผู้ป่วยกำลังเข้าสู่ระยะวิกฤตข...,อาการที่แสดงว่าผู้ป่วยกำลังเข้าสู่ระยะวิกฤตของ...,[{'title': 'โรคไข้เลือดออกมีสามระยะทางคลินิกคื...
98,327,DDDD,1,2,อะไรคือปัจจัยเสี่ยงที่ทำให้ผู้ป่วยไข้เลือดออกม...,ปัจจัยเสี่ยงที่ทำให้ผู้ป่วยไข้เลือดออกมีโอกาสเ...,[{'title': 'โรคไข้เลือดออกมีสามระยะทางคลินิกคื...


## Add retrieved facts

In [3]:
import json
import requests
from tqdm import tqdm

# enable tqdm for pandas
tqdm.pandas()

def get_facts(question):
    r = requests.get(
        f"{RETRIEVE_BASE_URL}{RETRIEVE_ENDPOINT}",
        params={"q": question},
        timeout=10,
    )
    r.raise_for_status()
    results = r.json()["results"]

    return [
        {"title": x["text"], "score": x["score"], "id": x["id"]}
        for x in results
    ]

data['retrieved_facts'] = data['questions'].progress_apply(get_facts)

  0%|          | 0/100 [00:00<?, ?it/s]

100%|██████████| 100/100 [00:13<00:00,  7.26it/s]


In [4]:
data.iloc[0]['retrieved_facts'][0]

{'title': 'การจัดการภาวะบกพร่องด้านการเรียนรู้ควรปรึกษาจิตแพทย์เฉพาะทางด้าน LD เพื่อรับการทดสอบและประเมินทางจิตวิทยา วินิจฉัยภาวะที่เกี่ยวข้อง และเสริมทักษะการอ่าน เขียน หรือคำนวณที่บกพร่องให้เด็กกลับมาเรียนรู้ได้อย่างปกติ',
 'score': 0.904296875,
 'id': 'article_67f7a1c39b542b3157bd50ac-002'}

## Add citations

In [5]:
import json

def get_citations(row):
    facts = eval(row["facts"])
    fact_ids = {d['id'] for d in facts}
    
    retrieved_facts = row["retrieved_facts"]
    retrieved_ids = [d["id"] for d in retrieved_facts]
    
    return [i+1 for i, fid in enumerate(retrieved_ids) if fid in fact_ids]


data["citations"] = data.apply(get_citations, axis=1)
data.head(100)

,conver_id,pattern,sampling_stergy,turn_ids,questions,answers,facts,retrieved_facts,citations
0,10,CCCC,1,1,ลูกมีปัญหาในการอ่านและเขียนผิดพลาดบ่อย ควรเริ่...,ควรเริ่มต้นด้วยการปรึกษาจิตแพทย์เฉพาะทางด้านคว...,[{'title': 'การจัดการภาวะบกพร่องด้านการเรียนรู...,[{'title': 'การจัดการภาวะบกพร่องด้านการเรียนรู...,"[1, 6]"
1,10,CCCC,1,2,การตรวจอัลตราซาวด์เต้านมสามารถใช้ตรวจพบภาวะใดไ...,การตรวจอัลตราซาวด์เต้านมสามารถใช้ตรวจพบภาวะต่า...,[{'title': 'อัลตราซาวนด์เต้านมมีบทบาทหลักในการ...,[{'title': 'อัลตร้าซาวด์สามารถตรวจอวัยวะใต้ผิว...,"[3, 4]"
2,10,CCCC,1,3,โรคงูสวัดเกิดจากเชื้อไวรัสชนิดใดที่กลับมาทำให้...,โรคงูสวัดเกิดจากเชื้อไวรัสวาริเซลลา-โซสเตอร์ (...,[{'title': 'โรคงูสวัดคือผื่นและตุ่มน้ำใสที่เกิ...,[{'title': 'โรคงูสวัดคือผื่นและตุ่มน้ำใสที่เกิ...,"[1, 2, 4]"
3,10,CCCC,1,4,อาการปวดศีรษะเรื้อรังสามารถเกี่ยวข้องกับภาวะหล...,อาการปวดศีรษะเรื้อรังสามารถเกี่ยวข้องกับภาวะหล...,[{'title': 'อาการสำคัญของโรคหลอดเลือดอักเสบจาก...,[{'title': 'อาการปวดหัวเรื้อรัง (เกิดมากกว่า 1...,"[5, 6]"
4,10,CCCC,1,5,ทารกแรกเกิดที่มีภาวะช็อกอาจแสดงอาการอย่างไรในช...,ทารกแรกเกิดที่มีภาวะช็อกอาจแสดงอาการอัตราการเต...,[{'title': 'โดยทั่วไปความดันโลหิตสูงในทารกแรกเ...,[{'title': 'โดยทั่วไปความดันโลหิตสูงในทารกแรกเ...,"[1, 2]"
...,...,...,...,...,...,...,...,...,...
95,320,DRPP,1,4,มีปัจจัยใดบ้างที่ทำให้ไอหลังเป็นหวัดยืดเยื้อเก...,ปัจจัยที่ทำให้ไอหลังเป็นหวัดยืดเยื้อเกิน 3 สัป...,[{'title': 'ไอกึ่งเฉียบพลันคืออาการไอที่มีระยะ...,[{'title': 'ภาวะไอหลังการติดเชื้อทางเดินหายใจส...,"[2, 3, 4, 5, 8]"
96,320,DRPP,1,5,อะไรคือสาเหตุหลักที่ทำให้ไอหลังเป็นหวัดไม่หายเ...,สาเหตุหลักที่ทำให้ไอหลังเป็นหวัดไม่หายเสียทีคื...,[{'title': 'ภาวะไอหลังเป็นหวัดเกิดจากการอักเสบ...,[{'title': 'ภาวะไอหลังเป็นหวัดเกิดจากการอักเสบ...,"[1, 2, 4, 5, 6]"
97,327,DDDD,1,1,อาการใดที่แสดงว่าผู้ป่วยกำลังเข้าสู่ระยะวิกฤตข...,อาการที่แสดงว่าผู้ป่วยกำลังเข้าสู่ระยะวิกฤตของ...,[{'title': 'โรคไข้เลือดออกมีสามระยะทางคลินิกคื...,[{'title': 'ระยะฟักตัวของไข้เลือดออกไวรัลอยู่ท...,"[2, 3, 6]"
98,327,DDDD,1,2,อะไรคือปัจจัยเสี่ยงที่ทำให้ผู้ป่วยไข้เลือดออกม...,ปัจจัยเสี่ยงที่ทำให้ผู้ป่วยไข้เลือดออกมีโอกาสเ...,[{'title': 'โรคไข้เลือดออกมีสามระยะทางคลินิกคื...,[{'title': 'เด็กอายุต่ำกว่า 1 ปีและผู้สูงอายุม...,"[1, 4]"


In [6]:
from datasets import Dataset, DatasetDict

ds = Dataset.from_pandas(data, preserve_index=False)

dataset = DatasetDict({
    # "train": None,
    "validation": ds,  # dev
    # "test": None,
})

dataset.push_to_hub("weerayut/iq-data", private=True)

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ? shards/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Processing Files (0 / 0): |          |  0.00B /  0.00B            

New Data Upload: |          |  0.00B /  0.00B            

No files have been modified since last commit. Skipping to prevent empty commit.


CommitInfo(commit_url='https://huggingface.co/datasets/weerayut/iq-data/commit/93426dde24a18bb68dfc783617a3ffd4f0d50561', commit_message='Upload dataset', commit_description='', oid='93426dde24a18bb68dfc783617a3ffd4f0d50561', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/weerayut/iq-data', endpoint='https://huggingface.co', repo_type='dataset', repo_id='weerayut/iq-data'), pr_revision=None, pr_num=None)

In [7]:
dataset

DatasetDict({
    validation: Dataset({
        features: ['conver_id', 'pattern', 'sampling_stergy', 'turn_ids', 'questions', 'answers', 'facts', 'retrieved_facts', 'citations'],
        num_rows: 100
    })
})